## Inference to models from experiment-8 (basecnn B0 models with original images and traditional image augmentation & transformation techniques)

In [ ]:
import sys
import glob
import itertools
import os
import random
import gc
import numpy as np
import pandas as pd
import math
import pprint
import matplotlib.pylab as plt
import seaborn as sns
sns.set(rc={"axes.titlesize":15, "axes.labelsize":9,"axes.titlepad":15,
            "axes.labelpad":12, "legend.fontsize":9,
            "legend.title_fontsize":9, "figure.titlesize":15,
            "axes.grid":False})

from sklearn.model_selection import train_test_split, GroupKFold
from PIL import Image
from tqdm.auto import tqdm

import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow.keras.backend as K
from kaggle_datasets import KaggleDatasets

print('TF version:', tf.__version__)
print('Hub version:', tfhub.__version__)
print('Physical devices:', tf.config.list_physical_devices())

In [ ]:
def seed_everything(SEED):
    os.environ['PYTHONHASHSEED']=str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
    
class Config:
    seed = 2021
    model_arch = "efficientnetv2-b0" #"efficientnetv2-s-21k-ft1k" ## Choose model architecture
    hub_type = "feature_vector"
    wandb_project = 'lmju-covid-inf8'
    dataset = "ljmusiimcovid19inf8"
    BATCH_SIZE = 32
    TEST_IMAGE_DIMS = (256, 256)
    trail_run = False
    
    seed_everything(seed)
    
cfg = Config()

In [ ]:
#test_df = pd.read_csv('/kaggle/input/ljmusiimcovid19infv2/ljmu-siim-covid19-inf1-v2/test_df.csv')
test_df = pd.read_csv('/kaggle/input/ljmusiimcovid19inf8/ljmu-siim-covid19-inf8/test_df.csv')

test_df['Negative'] = 0
test_df.loc[test_df.Covid == 0, 'Negative'] = 1
test_df.head(3)

In [ ]:
test_df.shape

In [ ]:
if cfg.trail_run == True:
    test_df = test_df.sample(10)
    print("\ntest_df")
    display(test_df.head(10))


In [ ]:
# %pip install -q wandb 
#!pip install --upgrade wandb
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_apikey") 
wandb.login(key=wandb_api)
print('wandb connected ...')

<span style="color: #00857e; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">Custom Wrapper for Loading TFHub Model trained in TPU</span>

Since the EffNetV2 Classifier models were trained on a TPU with the `tfhub.KerasLayer` formed with the handle argument as a GCS path, while loading the saved model for inference, the method tries to download the pre-trained weights from the definition of the layer from training i.e a GCS path.

Since, inference notebooks don't have GCS and internet access, it is not possible to load the model without the pretrained weights explicitly loaded from the local directory.

If the models were trained on a GPU, we can use the cache location method to load the pre-trained weights by storing them in a cache folder with the hashed key of the model location, as the folder name. I tried this method here but, it doesn't seem to work as the model was trained with a GCS path defined in the `tfhub.KerasLayer` and the method kept on hitting the GCS path rather than loading the weights from the cache location.

The only solution was to create a wrapper class to correct the handle argument to load the right pretrained weights explicitly from the local directory.

In [ ]:
def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode

def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

In [ ]:
# Get the TensorFlow Hub model URL
# MODEL_ARCH_PATH = f'/kaggle/input/efficientnetv2-tfhub-weight-files/tfhub_models/{cfg.model_arch}/{cfg.hub_type}'
DS_GCS_PATH = KaggleDatasets().get_gcs_path("efficientnetv2-tfhub-weight-files")
# MODEL_ARCH_PATH = f'{DS_GCS_PATH}/tfhub_models/{cfg.model_arch}/{cfg.hub_type}'
MODEL_ARCH_PATH = f'../input/efficientnetv2-tfhub-weight-files/tfhub_models/{cfg.model_arch}/{cfg.hub_type}'

# Get the GCS path of the images from the Kaggle dataset
# GCS_DS_PATH =  KaggleDatasets().get_gcs_path(cfg.dataset)+"/ljmu-siim-covid19-inf1/test/"
GCS_DS_PATH =  f'../input/ljmusiimcovid19inf8/ljmu-siim-covid19-inf8/test/'
MODEL_PATH = f'../input/ljmusiimcovid19inf8/ljmu-siim-covid19-inf8/models'

print(MODEL_ARCH_PATH)
print()
print(GCS_DS_PATH)

# Custom wrapper class to load the right pretrained weights explicitly from the local directory
class KerasLayerWrapper(tfhub.KerasLayer):
    def __init__(self, handle, **kwargs):
        handle = tfhub.KerasLayer(tfhub.load(MODEL_ARCH_PATH))
        super().__init__(handle, **kwargs)

<span style="color: #00857e; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">Predict Study Level</span>

In [ ]:
# strategy = auto_select_accelerator()
# BATCH_SIZE = strategy.num_replicas_in_sync * 16
test_paths = [GCS_DS_PATH+jpg for jpg in test_df['Id'].tolist()]
test_paths[0]

In [ ]:
#for predictions
label_cols = ['Covid', 'Negative']
pred_df = test_df[['Id', 'Covid', 'Negative']].copy()
pred_df[label_cols] = 0

test_decoder = build_decoder(with_labels=False,
                             target_size=(cfg.TEST_IMAGE_DIMS[0],
                                          cfg.TEST_IMAGE_DIMS[0]), ext='jpg')
test_dataset = build_dataset(
    test_paths, bsize=cfg.BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)


with tf.device('/device:GPU:0'):
    models = []
    models0 = tf.keras.models.load_model(f'{MODEL_PATH}/model0.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    
    models1 = tf.keras.models.load_model(f'{MODEL_PATH}/model1.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models2 = tf.keras.models.load_model(f'{MODEL_PATH}/model2.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models3 = tf.keras.models.load_model(f'{MODEL_PATH}/model3.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models4 = tf.keras.models.load_model(f'{MODEL_PATH}/model4.h5',
                                      custom_objects={'KerasLayer': KerasLayerWrapper})
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)

pred_df[label_cols] = sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)

del models
del models0, models1, models2, models3, models4
del test_dataset, test_decoder
gc.collect()

print("Prediction completed ...")

In [ ]:
pred_df['Covid_Prob'] = pred_df['Covid']
pred_df['Negative_Prob'] = pred_df['Negative']
# CHECK: sum of probs shouldn't be over 100
#df_100 = pred_df[pred_df['Covid_Prob']+pred_df['Negative_Prob']>100]
#df_100.head()

In [ ]:
pred_df['Covid'] = pred_df.apply(lambda x: 1 if x['Covid_Prob'] == x['Negative_Prob'] else x['Covid'], axis=1)
pred_df['Negative'] = pred_df.apply(lambda x: 0 if x['Covid_Prob'] == x['Negative_Prob'] else x['Negative'], axis=1)
pred_df['Covid'] = pred_df.apply(lambda x: 0 if x['Covid_Prob'] < 0.50 else 1, axis=1)
pred_df['Negative'] = pred_df.apply(lambda x: 0 if x['Negative_Prob'] < 0.50 else 1, axis=1)

#y_pred = (y_pred > 0.5) 
pred_df.head(10)

In [ ]:
test_df.head(10)

In [ ]:
#summary reports
#for metrics
x_test = test_df["Id"]
y_true_covid = test_df["Covid"]
y_true_negative = test_df["Negative"]

y_pred_covid = pred_df["Covid"]
y_pred_negative = pred_df["Negative"]

y_pred_covid_prob = pred_df["Covid_Prob"]
y_pred_negative_prob = pred_df["Negative_Prob"]

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# confusion matrix
# outcome values order in sklearn
# classification report for precision, recall f1-score and accuracy

print("\n ****** Covid Summary ****** ")
matrix = confusion_matrix(y_true_covid,y_pred_covid, labels=[1,0])
print('Confusion matrix : \n',matrix)
tp, fn, fp, tn = confusion_matrix(y_true_covid,y_pred_covid,labels=[1,0]).reshape(-1)
print('Outcome values : \n', "TP = "+str(tp), "FN = "+str(fn), "FP = "+str(fp), "TN = "+str(tn))
matrix = classification_report(y_true_covid,y_pred_covid,labels=[1,0])
print('Classification report : \n',matrix)

#print("\n ****** Negative Summary ****** ")
#matrix = confusion_matrix(y_true_negative,y_pred_negative, labels=[1,0])
#print('Confusion matrix : \n',matrix)
#tp, fn, fp, tn = confusion_matrix(y_true_negative,y_pred_negative,labels=[1,0]).reshape(-1)
#print('Outcome values : \n', "TP = "+str(tp), "FN = "+str(fn), "FP = "+str(fp), "TN = "+str(tn))
#matrix = classification_report(y_true_negative,y_pred_negative,labels=[1,0])
#print('Classification report : \n',matrix)

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

lr_probs =  pred_df[["Covid_Prob"]].to_numpy()
lr_precision, lr_recall, _ = precision_recall_curve(y_true_covid, lr_probs)
lr_f1, lr_auc = f1_score(y_true_covid, y_pred_covid), auc(lr_recall, lr_precision)
lr_auc = roc_auc_score(y_true_covid, lr_probs)
precision = precision_score(y_true_covid, y_pred_covid, labels=[1,0], average='macro')
recall = recall_score(y_true_covid, y_pred_covid, average='macro')

print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('ROC AUC: %.3f' % (lr_auc))
print('F1: %.3f AUC: %.3f' % (lr_f1, lr_auc))


In [ ]:
#plot graphs
from matplotlib import pyplot
from sklearn.metrics import roc_curve

lr_fpr, lr_tpr, _ = roc_curve(y_true_covid, lr_probs)
pyplot.plot(lr_fpr, lr_tpr, marker='.', label=str('ROC: %.3f' % (lr_auc)))
pyplot.title("Receiver Operating Characteristic")
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

pyplot.plot(lr_recall, lr_precision, marker='.', color='green', label=str('Precision: %.3f Recall: %.3f' % (precision, recall)))
pyplot.title("Precision-Recall")
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true_covid, y_pred_covid, labels=[1,0])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Covid','Negative'])
fig = pyplot.figure()
ax = fig.add_subplot(111)
#disp.title("Confusion Matrix")
disp = disp.plot(include_values=True, cmap='viridis',xticks_rotation='horizontal', ax=ax)
pyplot.show()

In [ ]:
# upload CM to wandb
config_dict = dict(vars(Config))
#jobconfig = f"inf1-Arch:{config_dict['model_arch']};seed:{config_dict['seed']};batchsize:{config_dict['BATCH_SIZE']};dataset:{config_dict['dataset']}"
wandb.init(project=cfg.wandb_project) #, name='inf1')#, config=config_dict)

wandb.sklearn.plot_confusion_matrix(y_true_covid, y_pred_covid.to_numpy(), ["Covid","Negative"])
#wandb.sklearn.plot_confusion_matrix(y_true_negative, y_pred_negative, ["Covid","Negative"])

#get probability of positive only
#prob_df = pred_df[["Covid_Prob","Negative_Prob"]]
#prob_df = pred_df[["Covid_Prob"]]

#wandb.sklearn.plot_precision_recall(y_true_covid, prob_df.to_numpy(), ["Covid","Negative"])
#wandb.sklearn.plot_precision_recall(y_true_negative, y_pred_negative_prob, ["Covid","Negative"])

#wandb.sklearn.plot_roc(y_true_covid, prob_df.to_numpy(), ["Covid","Negative"])
#wandb.sklearn.plot_roc(y_true_negative, y_pred_negative_prob, ["Covid","Negative"])

wandb.finish()

print("Report completed ...")